In [5]:
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import warnings
import time
from collections import Counter
warnings.filterwarnings("ignore")


plt.style.use("fivethirtyeight")

sns.set_style({'font.sans-serif': ['simsun', 'Arial']})
sns.set_style('darkgrid', {'font.sans-serif': ['simhei', 'Arial']})
%matplotlib inline
current_date = pd.to_datetime(
    time.strftime('%Y-%m-%d', time.localtime(time.time())))

res = {}
data_path = r'./Documents/赛题1数据集/'
current_date

Timestamp('2019-04-03 00:00:00')

In [44]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_in.loc[(df_in[col_name] < fence_low) == True,col_name]= df_in.loc[(df_in[col_name] < fence_low) == True,col_name].fillna(fence_low)
    df_in.loc[(df_in[col_name] > fence_high)== True,col_name]=df_in.loc[(df_in[col_name] > fence_high)== True,col_name].fillna(fence_high)



# 资产负债表

In [36]:
df_asset_bpk = pd.read_excel(data_path+r'上市信息财务信息资产负债表.xlsx')
df_asset = df_asset_bpk.copy()

In [37]:
df_asset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19620 entries, 0 to 19619
Data columns (total 24 columns):
企业编号               19620 non-null int64
标题                 19620 non-null object
日期                 19620 non-null object
资产:货币资金(元)         18862 non-null object
资产:应收账款(元)         18291 non-null object
资产:其它应收款(元)        18460 non-null object
资产:存货(元)           18115 non-null object
资产:流动资产合计(元)       18455 non-null object
资产:长期股权投资(元)       12873 non-null object
资产:累计折旧(元)         1446 non-null object
资产:固定资产(元)         18854 non-null object
资产:无形资产(元)         18595 non-null object
资产:资产总计(元)         18862 non-null object
负债:应付账款(元)         15646 non-null object
负债:预收账款(元)         18117 non-null object
负债:存货跌价准备(元)       1446 non-null object
负债:流动负债合计(元)       18455 non-null object
负债:长期负债合计(元)       17729 non-null object
负债:负债合计(元)         18862 non-null object
权益:实收资本(或股本)(元)    18851 non-null object
权益:资本公积金(元)        18731 non-null object
权益:盈余公积金(元)        18669 non-null

In [7]:
A = ['资产:货币资金(元)', '资产:长期股权投资(元)', '资产:固定资产(元)', '资产:无形资产(元)','负债:长期负债合计(元)','权益:实收资本(或股本)(元)',
     '资产:流动资产合计(元)','资产:资产总计(元)','负债:流动负债合计(元)','负债:负债合计(元)','权益:股东权益合计(元)']
df_asset=df_asset[~df_asset['权益:股东权益合计(元)'].isin(['正无穷大万亿'])]
df_asset1= pd.DataFrame({'企业编号':df_asset['企业编号'],'资产:货币资金(元)':df_asset['资产:货币资金(元)'],'资产:长期股权投资(元)': df_asset['资产:长期股权投资(元)'],
                         '资产:固定资产(元)': df_asset['资产:固定资产(元)'],'资产:无形资产(元)': df_asset['资产:无形资产(元)'],'负债:长期负债合计(元)':df_asset['负债:长期负债合计(元)'],
                         '权益:实收资本(或股本)(元)':df_asset['权益:实收资本(或股本)(元)'],'资产:流动资产合计(元)': df_asset['资产:流动资产合计(元)'],
                         '资产:资产总计(元)': df_asset['资产:资产总计(元)'],'负债:流动负债合计(元)': df_asset['负债:流动负债合计(元)'],'负债:负债合计(元)':df_asset['负债:负债合计(元)'],
                         '权益:股东权益合计(元)':df_asset['权益:股东权益合计(元)'],
})
df_asset1.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 19619 entries, 0 to 19619
Data columns (total 12 columns):
企业编号               19619 non-null int64
资产:货币资金(元)         18861 non-null object
资产:长期股权投资(元)       12872 non-null object
资产:固定资产(元)         18853 non-null object
资产:无形资产(元)         18594 non-null object
负债:长期负债合计(元)       17728 non-null object
权益:实收资本(或股本)(元)    18850 non-null object
资产:流动资产合计(元)       18454 non-null object
资产:资产总计(元)         18861 non-null object
负债:流动负债合计(元)       18454 non-null object
负债:负债合计(元)         18861 non-null object
权益:股东权益合计(元)       18853 non-null object
dtypes: int64(1), object(11)
memory usage: 1.9+ MB


In [8]:
for arr in A:
    chinese = set(df_asset1[arr].str.extract(
        "([^0-9\.\-]+)", expand=False).dropna())
    print(arr, chinese)
for arr in A:
    df_asset1.loc[(df_asset1[arr].str.contains("万") == True), arr] = df_asset1.loc[(
        df_asset1[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*0.0001
    df_asset1.loc[(df_asset1[arr].str.contains("亿") == True), arr] = df_asset1.loc[(
        df_asset1[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*1
for arr in A:
     df_asset1[arr]=pd.to_numeric( df_asset1[arr],errors='coerce')

资产:货币资金(元) {'亿', '万'}
资产:长期股权投资(元) {'亿', '万'}
资产:固定资产(元) {'亿', '万'}
资产:无形资产(元) {'亿', '万'}
负债:长期负债合计(元) {'亿', '万'}
权益:实收资本(或股本)(元) {'亿', '万'}
资产:流动资产合计(元) {'亿', '万'}
资产:资产总计(元) {'亿', '万'}
负债:流动负债合计(元) {'亿', '万'}
负债:负债合计(元) {'亿', '万'}
权益:股东权益合计(元) {'亿', '万'}


In [95]:
df_asset2=df_asset1.copy()

In [93]:
for arr in A:
    remove_outlier(df_asset2, arr)
df_asset2.describe()

,企业编号,资产:货币资金(元),资产:长期股权投资(元),资产:固定资产(元),资产:无形资产(元),负债:长期负债合计(元),权益:实收资本(或股本)(元),资产:流动资产合计(元),资产:资产总计(元),负债:流动负债合计(元),负债:负债合计(元),权益:股东权益合计(元)
count,19619.000000,18832.000000,16012.000000,1.883000e+04,1.871800e+04,1.827700e+04,18831.000000,18575.000000,18835.000000,18575.000000,18832.000000,18832.000000
mean,2502.105102,142.580616,295.986462,4.028220e+02,5.415636e+03,9.741457e+03,16.607059,46.650426,43.027878,50.380872,50.217855,46.774029
std,861.625393,6427.613022,4776.293636,2.290329e+04,6.442354e+05,8.009582e+05,74.591571,710.833953,634.801261,563.792521,549.503201,1073.311233
min,1001.000000,-99.944766,-427.729651,-1.000000e+02,-9.997863e+01,-1.502334e+02,-72.556880,-98.036005,-82.513013,-97.254836,-97.063776,-14251.488710
25%,1756.000000,-19.480172,-0.565971,-4.575285e+00,-3.831774e+00,-1.558658e+01,0.000000,0.259379,2.501147,-2.478294,-2.018390,2.500152
50%,2514.000000,4.950000,14.678937,2.699247e+00,1.428229e-01,2.670775e+00,0.000000,12.174544,11.078739,14.004754,11.808702,7.913549
75%,3240.500000,42.723265,58.615524,1.856327e+01,1.523740e+01,5.060255e+01,10.300000,31.994482,27.314823,41.263754,37.834987,21.124039
max,4000.000000,622866.080393,381473.529340,1.948040e+06,8.790694e+07,1.047902e+08,4939.714566,45279.835862,43345.620140,44971.460810,43868.417546,85332.932385


In [96]:
df_asset1.describe()

,企业编号,资产:货币资金(元),资产:长期股权投资(元),资产:固定资产(元),资产:无形资产(元),负债:长期负债合计(元),权益:实收资本(或股本)(元),资产:流动资产合计(元),资产:资产总计(元),负债:流动负债合计(元),负债:负债合计(元),权益:股东权益合计(元)
count,19619.000000,18832.000000,16012.000000,1.883000e+04,1.871800e+04,1.827700e+04,18831.000000,18575.000000,18835.000000,18575.000000,18832.000000,18832.000000
mean,2502.105102,142.580616,295.986462,4.028220e+02,5.415636e+03,9.741457e+03,16.607059,46.650426,43.027878,50.380872,50.217855,46.774029
std,861.625393,6427.613022,4776.293636,2.290329e+04,6.442354e+05,8.009582e+05,74.591571,710.833953,634.801261,563.792521,549.503201,1073.311233
min,1001.000000,-99.944766,-427.729651,-1.000000e+02,-9.997863e+01,-1.502334e+02,-72.556880,-98.036005,-82.513013,-97.254836,-97.063776,-14251.488710
25%,1756.000000,-19.480172,-0.565971,-4.575285e+00,-3.831774e+00,-1.558658e+01,0.000000,0.259379,2.501147,-2.478294,-2.018390,2.500152
50%,2514.000000,4.950000,14.678937,2.699247e+00,1.428229e-01,2.670775e+00,0.000000,12.174544,11.078739,14.004754,11.808702,7.913549
75%,3240.500000,42.723265,58.615524,1.856327e+01,1.523740e+01,5.060255e+01,10.300000,31.994482,27.314823,41.263754,37.834987,21.124039
max,4000.000000,622866.080393,381473.529340,1.948040e+06,8.790694e+07,1.047902e+08,4939.714566,45279.835862,43345.620140,44971.460810,43868.417546,85332.932385


In [49]:
res['资产负债表']=df_asset1

# 利润表

In [14]:
df_profit_bpk = pd.read_excel(data_path+r'上市信息财务信息-利润表.xlsx')
df_profit = df_profit_bpk.copy()
B=['营业成本(元)','销售费用(元)','财务费用(元)','管理费用(元)','投资收益(元)','营业利润(元)','利润总额(元)','归属母公司所有者净利润(元)']
for arr in B:
    chinese = set(df_profit[arr].str.extract(
        "([^0-9\.\-]+)", expand=False).dropna())
    print(arr, chinese)
for arr in B:
    df_profit.loc[(df_profit[arr].str.contains("万") == True), arr] = df_profit.loc[(
       df_profit[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*10000
    df_profit.loc[(df_profit[arr].str.contains("亿") == True), arr] = df_profit.loc[(
        df_profit[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*100000000
    df_profit[arr]=pd.to_numeric(df_profit[arr],errors='coerce')
df_profit1=df_profit.drop(['标题','日期','营业收入(元)','资产减值损失(元)','资产减值损失(元)','所得税(元)'],axis=1)
df_profit1.describe()
df_profit1.info()

营业成本(元) {'亿', '万'}
销售费用(元) {'亿', '万'}
财务费用(元) {'亿', '万'}
管理费用(元) {'亿', '万'}
投资收益(元) {'亿', '万'}
营业利润(元) {'亿', '万'}
利润总额(元) {'亿', '万'}
归属母公司所有者净利润(元) {'亿', '万'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621 entries, 0 to 19620
Data columns (total 9 columns):
企业编号              19621 non-null int64
营业成本(元)           19174 non-null float64
销售费用(元)           18717 non-null float64
财务费用(元)           19197 non-null float64
管理费用(元)           19621 non-null float64
投资收益(元)           17532 non-null float64
营业利润(元)           19613 non-null float64
利润总额(元)           19613 non-null float64
归属母公司所有者净利润(元)    19600 non-null float64
dtypes: float64(8), int64(1)
memory usage: 1.3 MB


In [16]:
df_profit1.describe()

,企业编号,营业成本(元),销售费用(元),财务费用(元),管理费用(元),投资收益(元),营业利润(元),利润总额(元),归属母公司所有者净利润(元)
count,19621.000000,1.917400e+04,1.871700e+04,1.919700e+04,1.962100e+04,1.753200e+04,1.961300e+04,1.961300e+04,1.960000e+04
mean,2502.233474,5.401517e+09,2.766819e+08,1.026169e+08,5.615918e+08,2.255908e+08,9.281487e+08,9.361126e+08,6.463964e+08
std,861.675321,3.843885e+10,1.526362e+09,4.818345e+08,4.583625e+09,2.716090e+09,8.200435e+09,8.234424e+09,6.356620e+09
min,1001.000000,-6.105094e+04,-1.588870e+07,-1.727756e+09,0.000000e+00,-1.527000e+09,-1.740845e+10,-1.746173e+10,-1.373285e+10
25%,1756.000000,2.988833e+08,1.722000e+07,2.314372e+05,4.084657e+07,5.856164e+05,2.940000e+07,3.210138e+07,1.445701e+07
50%,2514.000000,8.253113e+08,5.108828e+07,1.182130e+07,8.800422e+07,6.513276e+06,1.060784e+08,1.108827e+08,6.955887e+07
75%,3241.000000,2.528021e+09,1.606295e+08,5.691388e+07,2.154248e+08,3.591278e+07,3.483784e+08,3.560778e+08,2.344761e+08
max,4000.000000,1.890398e+12,6.606700e+10,2.164800e+10,1.777230e+11,1.559190e+11,3.618420e+11,3.646410e+11,2.839630e+11


In [17]:
res['利润表']=df_profit1

# 现金流量表

In [19]:
df_cash_bpk = pd.read_excel(data_path+r'上市信息财务信息-现金流量表.xlsx')
df_cash = df_cash_bpk.copy()

In [106]:
df_cash.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621 entries, 0 to 19620
Data columns (total 27 columns):
企业编号                               19621 non-null int64
标题                                 19621 non-null object
日期                                 19621 non-null object
经营:销售商品、提供劳务收到的现金(元)               19191 non-null object
经营:收到的税费返还(元)                      13718 non-null object
经营:收到其他与经营活动有关的现金(元)               19602 non-null object
经营:经营活动现金流入小计(元)                   19621 non-null object
经营:购买商品、接受劳务支付的现金(元)               19160 non-null object
经营:支付给职工以及为职工支付的现金(元)              19620 non-null object
经营:支付的各项税费(元)                      19620 non-null object
经营:支付其他与经营活动有关的现金(元)               19620 non-null object
经营:经营活动现金流出小计(元)                   19621 non-null object
经营:经营活动产生的现金流量净额(元)                19621 non-null object
投资:取得投资收益所收到的现金(元)                 14283 non-null object
投资:处置固定资产、无形资产和其他长期资产收回的现金净额(元)    16016 non-null object
投资:投资活动现金流入小计(元)                   18

In [20]:
df_cash1=pd.DataFrame({'企业编号':df_cash['企业编号'],'经营流入':df_cash['经营:经营活动现金流入小计(元)'],
                '经营净额':df_cash['经营:经营活动产生的现金流量净额(元)'], '投资流出':df_cash['投资:投资活动现金流出小计(元)'], '投资净额':df_cash['投资:投资活动产生的现金流量净额(元)'],
                  '筹资流入':df_cash['筹资:筹资活动现金流入小计(元)'],'筹资净额':df_cash['筹资活动产生的现金流量净额(元)'],})
C=['经营流入','经营净额','投资流出','投资净额', '筹资流入','筹资净额']
for arr in C:
    chinese = set(df_cash1[arr].str.extract(
        "([^0-9\.\-]+)", expand=False).dropna())
    print(arr, chinese)
for arr in C:
    df_cash1.loc[(df_cash1[arr].str.contains("万") == True), arr] = df_cash1.loc[(
        df_cash1[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*0.0001
    df_cash1.loc[(df_cash1[arr].str.contains("亿") == True), arr] = df_cash1.loc[(
        df_cash1[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*1
    df_cash1[arr]=pd.to_numeric(df_cash1[arr],errors='coerce')
df_cash1.info()

经营流入 {'亿', '万'}
经营净额 {'亿', '万'}
投资流出 {'亿', '万'}
投资净额 {'亿', '万'}
筹资流入 {'亿', '万'}
筹资净额 {'亿', '万'}
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19621 entries, 0 to 19620
Data columns (total 7 columns):
企业编号    19621 non-null int64
经营流入    19621 non-null float64
经营净额    19621 non-null float64
投资流出    19550 non-null float64
投资净额    19566 non-null float64
筹资流入    17268 non-null float64
筹资净额    19066 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.0 MB


In [21]:
df_cash1.describe()

,企业编号,经营流入,经营净额,投资流出,投资净额,筹资流入,筹资净额
count,19621.000000,1.962100e+04,1.962100e+04,1.955000e+04,1.956600e+04,1.726800e+04,1.906600e+04
mean,2502.233474,1.152066e+10,3.832527e+08,1.013457e+10,-8.190982e+08,7.633609e+09,4.710430e+08
std,861.675321,9.364129e+10,1.954707e+10,1.280162e+11,1.423634e+10,9.604123e+10,6.331283e+09
min,1001.000000,-3.431075e+10,-3.758490e+11,-3.399501e+08,-5.179800e+11,-3.372392e+09,-1.024840e+11
25%,1756.000000,3.923690e+08,-6.506988e+07,5.901900e+07,-3.391531e+08,8.540409e+07,-8.208508e+07
50%,2514.000000,1.144482e+09,1.461882e+07,3.061123e+08,-7.391943e+07,4.693500e+08,3.021000e-01
75%,3241.000000,3.583944e+09,1.901435e+08,1.075447e+09,-4.530000e+00,1.814397e+09,2.183738e+08
max,4000.000000,3.477693e+12,8.829020e+11,5.930847e+12,3.914480e+11,7.023257e+12,2.933490e+11


In [119]:
res['现金流量表']=df_cash1

# 每股指标

In [59]:
df_meiguzhibiao_bpk = pd.read_excel(data_path+r'上市公司财务信息-每股指标.xlsx')
df_meiguzhibiao = df_meiguzhibiao_bpk.copy()
df_meiguzhibiao1=df_meiguzhibiao.drop(['标题','日期','稀释每股收益(元)'],axis=1)
D=['基本每股收益(元)','扣非每股收益(元)','每股净资产(元)','每股公积金(元)','每股未分配利润(元)','每股经营现金流(元)']
for arr in D:
    df_meiguzhibiao1[arr]=pd.to_numeric(df_meiguzhibiao1[arr],errors='coerce')
df_meiguzhibiao1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 7 columns):
企业编号          28887 non-null int64
基本每股收益(元)     28824 non-null float64
扣非每股收益(元)     28073 non-null float64
每股净资产(元)      25597 non-null float64
每股公积金(元)      25456 non-null float64
每股未分配利润(元)    25586 non-null float64
每股经营现金流(元)    25613 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


In [60]:
df_meiguzhibiao1.describe()

,企业编号,基本每股收益(元),扣非每股收益(元),每股净资产(元),每股公积金(元),每股未分配利润(元),每股经营现金流(元)
count,28887.000000,28824.000000,28073.000000,25597.000000,25456.000000,25586.000000,25613.000000
mean,2501.248693,0.270260,0.265088,4.599067,1.910476,1.437674,0.107034
std,862.606020,0.502749,0.484692,3.179353,1.784597,2.105874,1.827665
min,1001.000000,-6.830000,-6.830000,-16.747500,-1.271800,-19.265000,-95.583800
25%,1753.000000,0.045375,0.043900,2.626600,0.676525,0.508700,-0.133500
50%,2512.000000,0.151700,0.150000,3.982600,1.478100,1.102550,0.067800
75%,3241.000000,0.370000,0.370000,5.791800,2.572600,2.023225,0.362300
max,4000.000000,21.560000,21.560000,81.491600,23.764500,70.465200,96.179800


In [129]:
res['每股指标']=df_meiguzhibiao1

# 成长能力指标

In [62]:
df_growth_bpk = pd.read_excel(data_path+r'上市信息财务信息-成长能力指标.xlsx')
df_growth= df_growth_bpk.copy()
df_growth=df_growth.drop(['标题','日期','扣非净利润(元)','归属净利润同比增长(元)','营业总收入滚动环比增长(元)', '归属净利润滚动环比增长(元)', '扣非净利润滚动环比增长(元)'],axis=1)
E = ['营业总收入(元)', '毛利润(元)', '归属净利润(元)']
for arr in E:
    df_growth.loc[(df_growth[arr].str.contains("万亿") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("万亿") == True), arr].str.replace('万亿', '').astype(float)*1000000000000
    df_growth.loc[(df_growth[arr].str.contains("万") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("万") == True), arr].str.replace('万', '').astype(float)*10000
    df_growth.loc[(df_growth[arr].str.contains("亿") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("亿") == True), arr].str.replace('亿', '').astype(float)*100000000
F = ['营业总收入同比增长(元)', '扣非净利润同比增长(元)']
for arr in F:
    df_growth[arr] = df_growth[arr].replace('--%', np.nan)
for arr in F:
    df_growth.loc[(df_growth[arr].str.contains("%") == True), arr] = df_growth.loc[(
        df_growth[arr].str.contains("%") == True), arr].str.replace(',', '').str.rstrip('%').astype(float)/100
df_growth = df_growth.convert_objects(convert_numeric=True)

In [63]:
df_growth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 6 columns):
企业编号            28887 non-null int64
营业总收入(元)        28877 non-null float64
毛利润(元)          28185 non-null float64
归属净利润(元)        28887 non-null float64
营业总收入同比增长(元)    28229 non-null float64
扣非净利润同比增长(元)    22907 non-null float64
dtypes: float64(5), int64(1)
memory usage: 1.3 MB


In [64]:
df_growth.describe()

,企业编号,营业总收入(元),毛利润(元),归属净利润(元),营业总收入同比增长(元),扣非净利润同比增长(元)
count,28887.000000,2.887700e+04,2.818500e+04,2.888700e+04,28229.000000,22907.000000
mean,2501.248693,7.689565e+09,1.074424e+09,6.726545e+08,3.383525,0.688314
std,862.606020,4.962548e+10,5.216009e+09,6.445600e+09,420.125627,63.671590
min,1001.000000,-3.410000e+04,-1.100000e+10,-1.610000e+10,-1.002000,-6706.238800
25%,1753.000000,4.550000e+08,1.100000e+08,2.289000e+07,0.001100,-0.203400
50%,2512.000000,1.200000e+09,2.810000e+08,8.092000e+07,0.131900,0.112900
75%,3241.000000,3.430000e+09,7.400000e+08,2.560000e+08,0.315800,0.458600
max,4000.000000,2.360000e+12,2.356000e+11,2.860000e+11,69980.822800,4768.861600


In [136]:
res['成长能力']=df_growth

# 盈利能力指标

In [67]:
df_abilityofprofit_bpk= pd.read_excel(data_path+r'上市信息财务信息盈利能力指标.xlsx')
df_abilityofprofit= df_abilityofprofit_bpk.copy()
G=['加权净资产收益率(%)','摊薄净资产收益率(%)','摊薄总资产收益率(%)','毛利率(%)','净利率(%)','实际税率(%)']
for arr in G:
        df_abilityofprofit[arr] = df_abilityofprofit[arr].replace('--%', np.nan)
        df_abilityofprofit[arr] =df_abilityofprofit[arr].str.rstrip('%').astype(float)/100
df_abilityofprofit.convert_objects(convert_numeric=True)
df_abilityofprofit=df_abilityofprofit.drop(['标题','日期'],axis=1)
df_abilityofprofit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 7 columns):
企业编号           28887 non-null int64
加权净资产收益率(%)    28218 non-null float64
摊薄净资产收益率(%)    28316 non-null float64
摊薄总资产收益率(%)    28887 non-null float64
毛利率(%)         28203 non-null float64
净利率(%)         28875 non-null float64
实际税率(%)        25648 non-null float64
dtypes: float64(6), int64(1)
memory usage: 1.5 MB


In [68]:
df_abilityofprofit.describe()

,企业编号,加权净资产收益率(%),摊薄净资产收益率(%),摊薄总资产收益率(%),毛利率(%),净利率(%),实际税率(%)
count,28887.000000,28218.000000,28316.000000,28887.000000,28203.000000,28875.000000,25648.000000
mean,2501.248693,0.045087,0.017108,0.031317,0.288438,0.033295,0.217841
std,862.606020,0.270053,2.145506,0.055255,0.200673,3.076402,1.662228
min,1001.000000,-12.905100,-288.947900,-1.431600,-7.795300,-327.525800,-42.760200
25%,1753.000000,0.014000,0.014000,0.006900,0.165500,0.028000,0.132600
50%,2512.000000,0.039700,0.038400,0.020800,0.258400,0.076200,0.169700
75%,3241.000000,0.082600,0.077300,0.046700,0.376800,0.144950,0.241700
max,4000.000000,16.140700,30.502000,0.837300,1.039000,68.058800,155.755700


In [142]:
res['盈利能力能力']=df_abilityofprofit

# 财务风险指标

In [69]:
df_risk_bpk= pd.read_excel(data_path+r'上市信息财务信息-财务风险指标.xlsx')
df_risk= df_risk_bpk.copy()
H=['流动比率','速动比率']
for arr in H:
      df_risk[arr] = pd.to_numeric(df_risk[arr],errors='coerce')
I=['资产负债率(%)','流动负债/总负债(%)']
for arr in I:
     df_risk[arr] = df_risk[arr].replace('--%', np.nan)
df_risk.convert_objects(convert_numeric=True)
for arr in I:
    df_risk[arr] =df_risk[arr].str.rstrip('%').astype(float)/100
df_risk.convert_objects(convert_numeric=True)
df_risk=df_risk.drop(['标题','日期','速动比率'],axis=1)
df_risk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 4 columns):
企业编号           28887 non-null int64
资产负债率(%)       28437 non-null float64
流动负债/总负债(%)    28355 non-null float64
流动比率           27703 non-null float64
dtypes: float64(3), int64(1)
memory usage: 902.8 KB


In [70]:
df_risk.describe()

,企业编号,资产负债率(%),流动负债/总负债(%),流动比率
count,28887.000000,28437.000000,28355.000000,27703.000000
mean,2501.248693,0.436883,0.793894,2.433465
std,862.606020,0.224050,0.217458,3.027833
min,1001.000000,0.000000,0.000000,0.010000
25%,1753.000000,0.263200,0.703050,1.160000
50%,2512.000000,0.421900,0.863400,1.670000
75%,3241.000000,0.586800,0.957700,2.670000
max,4000.000000,5.608900,1.168900,103.220000


In [155]:
res['财务风险能力']=df_risk

# 运营能力指标

In [78]:
df_operation_bpk= pd.read_excel(data_path+r'上市信息财务信息运营能力指标.xlsx')
df_operation= df_operation_bpk.copy()
J=['应收账款周转天数(天)','存货周转天数(天)']
for arr in J:
    df_operation[arr]=pd.to_numeric(df_operation[arr],errors='coerce')
df_operation.info()
df_operation=df_operation.drop(['标题','日期'],axis=1)
df_operation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 6 columns):
企业编号           28887 non-null int64
标题             28887 non-null object
日期             28887 non-null object
总资产周转率(次)      28797 non-null float64
应收账款周转天数(天)    27922 non-null float64
存货周转天数(天)      27776 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 1.3+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28887 entries, 0 to 28886
Data columns (total 4 columns):
企业编号           28887 non-null int64
总资产周转率(次)      28797 non-null float64
应收账款周转天数(天)    27922 non-null float64
存货周转天数(天)      27776 non-null float64
dtypes: float64(3), int64(1)
memory usage: 902.8 KB


In [79]:
df_operation.describe()

,企业编号,总资产周转率(次),应收账款周转天数(天),存货周转天数(天)
count,28887.000000,28797.000000,27922.000000,2.777600e+04
mean,2501.248693,0.411953,123.836809,1.343349e+03
std,862.606020,0.432501,1595.482425,1.538336e+05
min,1001.000000,0.000000,-35637.370000,-1.131990e+06
25%,1753.000000,0.150000,28.400000,4.787000e+01
50%,2512.000000,0.300000,66.350000,9.529000e+01
75%,3241.000000,0.540000,122.507500,1.914225e+02
max,4000.000000,10.090000,176659.700000,2.559399e+07


In [81]:
res['财务风险能力']=df_operation